LightGBM 의 Python 패키지는 pip install 로 설치가 가능합니다. 10k 데이터는 classifier comparison 튜토리얼에서 이용할 것이기 때문에 이번에는 24 만여 문장으로 이뤄진 데이터를 이용합니다.

```
pip install lightgbm
```

더 자세한 설치 방법은 공식 문서 [https://lightgbm.readthedocs.io/](https://lightgbm.readthedocs.io/) 를 참고하세요.

In [1]:
import numpy as np
import lightgbm as lgb
from lovit_textmining_dataset.navermovie_comments import load_sentiment_dataset

print(lgb.__version__)

texts, x, y, idx_to_vocab = load_sentiment_dataset(data_name='small', tokenize='komoran')
print(x.shape)
print(np.unique(y))

2.3.1
(239708, 10178)
[-1  1]


`x` 는 단어 빈도 벡터이므로 dtype 이 numpy.int64 입니다. 하지만 LightGBM 의 학습데이터의 dtype 은 float32 를 가정합니다.

In [2]:
x.dtype

dtype('int64')

그렇기 때문에 dtype 을 변환해 줍니다.

In [3]:
import scipy
x = scipy.sparse.csr_matrix(x, dtype=np.float32)
x.dtype

dtype('float32')

이 데이터의 label 은 negative 를 -1 로, positive 를 1 로 지정하였습니다만, LightGBM 는 [0, 1] 의 class label 만 입력가능합니다. -1 인 부분을 찾아 모두 0 으로 바꿔줍니다.

In [4]:
# label used in xgboost must be [0, 1]
y[np.where(y == -1)[0]] = 0
np.unique(y)

array([0, 1])

학습 정확도와 테스트 정확도를 확인하기 위하여 24 만여 문장을 9:1 로 학습용과 테스트용으로 나눕니다.

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

LightGBM 의 입력데이터는 Dataset 형식이며, data 에는 scipy.sparse.csr_matrix 와 같은 sparse matrix 및 numpy.ndarray 와 같은 dense matrix 도 입력할 수 있습니다. 학습용 데이터 `dtrain` 를 Dataset 으로 만듭니다. 하지만 predict 함수에 

In [6]:
dtrain = lgb.Dataset(data=x_train, label=y_train)

모델 학습에 이용할 패러매터를 설정합니다. binary classification 이기 때문에 이 값도 지정합니다. LightGBM 은 설정할 수 있는 패러매터가 많은데, 이들에 대해서는 official document 를 살펴보시기 바랍니다.

- https://lightgbm.readthedocs.io/en/latest/Parameters.html

In [7]:
param = {
    'num_leaves':31,
    # 'max_depth':8,
    'min_data_in_leaf': 10,
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbosity': 2
}

`num_round` 는 앙상블 모델을 구성하는 개별 모델의 개수입니다. 모델의 학습이 끝나면 학습된 모델 객체가 return 됩니다.

In [8]:
num_round = 10
bst = lgb.train(param, dtrain, num_round)

학습된 모델은 저장할 수 있으며,

In [9]:
bst.save_model('lgb.model')

load_model 을 이용하여 불러들일 수도 있습니다.

In [10]:
loaded_bst = lgb.Booster(model_file='lgb.model')

정확도를 측정하는 함수는 여러 번 이용할 것이기 때문에 accuracy 라는 함수로 만듭니다. predict 함수는 positive class 에 속할 확률을 return 합니다. 이 값이 0.5 이상인 값들을 모두 1 로, 그렇지 않은 값을 0 으로 만들어 label 을 얻습니다.

학습 데이터의 정확도와 테스트 데이터의 정확도를 각각 구합니다.

In [11]:
def accuracy(bst, x, y):
    pred_score = bst.predict(x)
    y_pred = np.zeros(pred_score.shape[0])
    y_pred[np.where(pred_score > 0.5)[0]] = 1
    n_correct = np.where(y_pred == y)[0].shape[0]
    accuracy = n_correct / y.shape[0]
    return accuracy

print('train accuracy : {}'.format(accuracy(loaded_bst, x_train, y_train)))
print('test  accuracy : {}'.format(accuracy(loaded_bst, x_test, y_test)))

train accuracy : 0.778763030912639
test  accuracy : 0.7778982937716408


이번에는 10 개가 아닌, 1000 개의 모델을 이용하여 앙상블 모델을 구축합니다. 정확도가 많이 올라갔습니다.

In [12]:
num_round = 1000
bst = lgb.train(param, dtrain, num_round)
bst.save_model('lgb1000.model')

print('train accuracy : {}'.format(accuracy(bst, x_train, y_train)))
print('test  accuracy : {}'.format(accuracy(bst, x_test, y_test)))

train accuracy : 0.9155360462044063
test  accuracy : 0.8932877226648868


그런데 Booster 는predict_proba 함수가 없어서 새로 classifier 를 학습합니다.

In [13]:
num_round = 1000
classifier = lgb.LGBMClassifier(
    boosting_type = 'gbdt',
    n_estimators = num_round,
    objective = 'binary',
    num_leaves = 31,
    min_child_samples = 10,
    n_jobs = 4,
    silent = False,
    reg_alpha = 0, # L1 regularization
    reg_lambda = 0, # L2 regularization
)

classifier = classifier.fit(x_train, y_train)
print(classifier)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=4, num_leaves=31, objective='binary',
               random_state=None, reg_alpha=0, reg_lambda=0, silent=False,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


Prediction probability 의 평균을 확인하면 학습, 테스트 모두 0.87 의 수준을 보입니다. 이는 LightGBM 은 Random Forest 와 달리 분류를 명확히 하는 모델임을 의미합니다.

In [14]:
def check_average_prediction_probability(classifier, x):
    proba = classifier.predict_proba(x)
    mean_proba = proba.max(axis=1).mean()
    return mean_proba

print('average prection probability with train data = {}'.format(
    check_average_prediction_probability(classifier, x_train)))
print('average prection probability with test data = {}'.format(
    check_average_prediction_probability(classifier, x_test)))

average prection probability with train data = 0.8724915968403639
average prection probability with test data = 0.8714802039472108
